In [21]:
from maskmm.ipstartup import *
from maskmm.kerasloss import *
from maskmm.tracker import *
from maskmm.utils.utils import pad
tracker.__init__("keras1", enabled=True, tolerance=0, basename="l0", \
                exclude_vars=("build_rpn_targets.config"))
tracker.clear()
from maskmm.config import Config
config = Config()

def t(name):
    """ load pytorch tensor and convert to tensorflow """
    x = load0(name)
    if name.startswith("mrcnn_class."):
        x = pad(x, 200)
        x = x.unsqueeze(0)
    return tf.convert_to_tensor(x.cpu().detach().numpy())

def getruns(*args):
    """ prepare arguments for test """
    runs = []
    for n in range(3):
        run = []
        for arg in args:
            data = t(f"{arg}_{n}")
            run.append(data)
        runs.append(run)
    return runs
    byvar = list(zip(*runs))
    stacked = []
    for var in byvar:
        stacked.append(K.stack([K.squeeze(item,0) for item in var]))
    runs.append(stacked)
    return runs

[root:INFO]:starting (cellevents.py:36, time=20:53)


time: 19.6 ms


In [2]:
# rpn_class_loss input shapes and return value
runs = getruns("rpn_class.rpn_match", "rpn_class.rpn_class_logits")
for run in runs:
    log.info([r.shape.as_list() for r in run])
    d(rpn_class_loss_graph(*run))

[root:INFO]:starting (cellevents.py:36, time=20:24)
[root:INFO]:[[1, 261888], [1, 261888, 2]] (<ipython-input-2-3d7215543a2b>:4, time=20:24)


0.0706761

[root:INFO]:[[1, 261888], [1, 261888, 2]] (<ipython-input-2-3d7215543a2b>:4, time=20:24)


2.4674687

[root:INFO]:[[1, 261888], [1, 261888, 2]] (<ipython-input-2-3d7215543a2b>:4, time=20:24)


0.012074454

[root:INFO]:[[3, 261888], [3, 261888, 2]] (<ipython-input-2-3d7215543a2b>:4, time=20:24)


0.85007304

time: 430 ms


In [3]:
runs = getruns("rpn_bbox.target_bbox", "rpn_bbox.rpn_match", "rpn_bbox.rpn_bbox")
for run in runs:
    log.info([r.shape.as_list() for r in run])

    d(rpn_bbox_loss_graph(config, *run))

[root:INFO]:starting (cellevents.py:36, time=20:24)
[root:INFO]:[[1, 256, 4], [1, 261888], [1, 261888, 4]] (<ipython-input-3-76bc841d48f7>:3, time=20:24)


0.16219305

[root:INFO]:[[1, 256, 4], [1, 261888], [1, 261888, 4]] (<ipython-input-3-76bc841d48f7>:3, time=20:24)


2.3060346

[root:INFO]:[[1, 256, 4], [1, 261888], [1, 261888, 4]] (<ipython-input-3-76bc841d48f7>:3, time=20:24)


0.094461724

[root:INFO]:[[3, 256, 4], [3, 261888], [3, 261888, 4]] (<ipython-input-3-76bc841d48f7>:3, time=20:24)


1.7550664

time: 53.6 ms


In [22]:
##### MASKMM DOES NOT MATCH THESE FOR IMAGE 0 or 1
##### input 0 is 157 but has trailing zeros
##### image 2 has no predictions????
 sparse_softmax_cross_entropy_with_logits versus nll_loss

active_class_ids = tf.convert_to_tensor(np.array([[0,1]]).astype(np.float32))
runs = getruns("mrcnn_class.target_class_ids", "mrcnn_class.pred_class_logits")
for run in runs:
    log.info([r.shape.as_list() for r in run])
    d(mrcnn_class_loss_graph(*run, active_class_ids))

[root:INFO]:starting (cellevents.py:36, time=20:53)
[root:INFO]:[[1, 200], [1, 200, 2]] (<ipython-input-22-ba712c8c12b4>:8, time=20:53)


0.774633

[root:INFO]:[[1, 200], [1, 200, 2]] (<ipython-input-22-ba712c8c12b4>:8, time=20:53)


0.728039

[root:INFO]:[[1, 200], [1, 200, 2]] (<ipython-input-22-ba712c8c12b4>:8, time=20:53)


nan

time: 51.8 ms


In [5]:
# pytorch class loss - different result including number for n=2
from maskmm.loss import *
d(mrcnn_class(load0("mrcnn_class.target_class_ids_0"), load0("mrcnn_class.pred_class_logits_0")))
d(mrcnn_class(load0("mrcnn_class.target_class_ids_1"), load0("mrcnn_class.pred_class_logits_1")))
d(mrcnn_class(load0("mrcnn_class.target_class_ids_2"), load0("mrcnn_class.pred_class_logits_2")))

[root:INFO]:starting (cellevents.py:36, time=20:45)
[root:WARNING]:not comparable mrcnn_class.target_class_ids_1 * cuda Tensor float32 [157] * cuda Tensor float32 [200] (tracker.py:244, time=20:45)
[root:WARNING]:not comparable mrcnn_class.pred_class_logits_1 * cuda Tensor float32 [157, 2] * cuda Tensor float32 [200, 2] (tracker.py:244, time=20:45)
[root:WARNING]:content unequal mrcnn_class.r0_1=3e-02 (tracker.py:238, time=20:45)


tensor(0.7665, grad_fn=<NllLossBackward>)

[root:INFO]:matched mrcnn_class.target_class_ids_2 (tracker.py:242, time=20:45)
[root:WARNING]:content unequal mrcnn_class.pred_class_logits_2=4e+02 (tracker.py:238, time=20:45)
[root:WARNING]:content unequal mrcnn_class.r0_2=1e-04 (tracker.py:238, time=20:45)


tensor(0.5912, grad_fn=<NllLossBackward>)

[root:WARNING]:not comparable mrcnn_class.target_class_ids_3 * cuda Tensor float32 [200] * cuda Tensor float32 [151] (tracker.py:244, time=20:45)
[root:WARNING]:not comparable mrcnn_class.pred_class_logits_3 * cuda Tensor float32 [200, 2] * cuda Tensor float32 [151, 2] (tracker.py:244, time=20:45)
[root:WARNING]:content unequal mrcnn_class.r0_3=4e-02 (tracker.py:238, time=20:45)


tensor(0.5810, grad_fn=<NllLossBackward>)

time: 151 ms
